In [1]:
#enveloping potential
# simple Example plot Enveloped Potential with two Harmonic Oscilators
##Imports:
import os, sys as csys, datetime, tempfile
import math, numpy as np, pandas as pd
import scipy.constants as const

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


csys.path.append(os.getcwd()+"/../../..")

#Ensembler
import ensembler.potentials.TwoD as pot2D

from ensembler.samplers.stochastic import metropolisMonteCarloIntegrator
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.system.basic_system import system

import ensembler.visualisation.plotPotentials as exPlot

%matplotlib inline

In [2]:
#ENERGIES Sampling
def write_out_etraj(traj, out_path, V1, V2):
    visited_positions =  np.array(list(map(np.array, traj.position)))
    state1 = V1.ene(visited_positions)
    state2 = V2.ene(visited_positions)
    Vrenergies = np.array(traj.total_potential_energy)

    out_file = open(out_path, "w")
    out_file.write("t\tV1\tV2\tVr\n")#HEADER
    for t, V1, V2, Vr in zip(traj.index, state1, state2, Vrenergies):
        out_file.write(str(t)+"\t"+str(V1)+"\t"+str(V2)+"\t"+str(Vr)+"\n")

    out_file.close()

In [3]:
    
#METROPOLIS CRITERION
##random part of Metropolis Criterion:
_defaultRandomness = lambda V1, V2: (np.random.rand() <= np.exp(-1.0 / (const.gas_constant / 1000.0 * 298) * (V1 - V2))) #pseudocount  for equal energies
##default Metropolis Criterion
_defaultMetropolisCriterion = lambda V1, V2: (V1 < V2 or _defaultRandomness(V1, V2))

    
def MC_exchange(sys1, sys2):
    s1 = sys1.potential.s
    s2 = sys2.potential.s

    V1_s1 = sys1.potential.ene(sys1._currentPosition)

    V2_s2 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s2
    V1_s2 = sys1.potential.ene(sys1._currentPosition)

    sys2.potential.s = s1
    V2_s1 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s1
    sys2.potential.s = s2

    exchange = _defaultMetropolisCriterion((V2_s1+V1_s2),(V1_s1+V2_s2))
    return exchange

In [4]:
#Simulation Setup
##Build System
periodic_bound = periodicBoundaryCondition(boundary=[[-180,180], [-180,180]])
integrator = metropolisMonteCarloIntegrator(fixed_step_size=[1,1], max_iteration_tillAccept=1000)


##STEPS
replicas = 10
each_sim = 200 #steps between trials
trials = 1000  #exchange attempts

#s_est = 0.609/Delta U_barrier
optimal_ses = {2.5: 0.122,
               5: 0.061,
               10: 0.03,
               20: 0.03, #0.015
               50: 0.006}

initial_position = None



In [5]:
#run multiple replicas
root_tmp_dir = tempfile.gettempdir()+"/edsSim"
if(not os.path.exists(root_tmp_dir)):
    os.mkdir(root_tmp_dir)
os.chdir(root_tmp_dir)

tmp_dir = root_tmp_dir+"/re_eds_simulation"
if(not os.path.exists(tmp_dir)):
    os.mkdir(tmp_dir)
os.chdir(tmp_dir)

print(tmp_dir)


for barrier in tqdm(optimal_ses, desc="Barrier: "):
    approach_dir = tmp_dir+"/simulation_with_"+str(barrier)+"kt_barrier"
    if(not os.path.exists(approach_dir)): os.mkdir(approach_dir)


    start_t = datetime.datetime.now()
    
    #build potential
    amplitude = barrier/2
    shift  = 90#np.rad2deg(0.5*math.pi)
    Eoff = [0,0]
    simulation_space =((-180,180), (-180,180))

    V1 = pot2D.wavePotential(phase_shift=(shift,shift), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude), radians=False)
    V2 = pot2D.wavePotential(phase_shift=(0.0, 0.0), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude), radians=False)
    edsPot1 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    edsPot2 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    
    
    start=0
    syst = None
    for replica in tqdm(range(start, replicas), desc="Replicas: "):
        ##INIT REPLICA
        ###fILE hANDLING
        replica_out = approach_dir+"/replica_"+str(replica)
        if(not os.path.exists(replica_out)):
            os.mkdir(replica_out)

        ###Build System
        sys1=system(potential=edsPot1, sampler=integrator, conditions=[periodic_bound])
        sys2=system(potential=edsPot2, sampler=integrator, conditions=[periodic_bound])

        sys1.potential.s = 0.5
        sys2.potential.s = 0.01

        ## SIMULATE REPLICA
        start_sim = datetime.datetime.now()
        s_val_posDict = {}  #OUT
        current_s_index = 0
        current_s_direction = 1

        for trial in tqdm(range(trials), desc="Current Simulation of replica "+str(replica)+": ",):
            cur_state1 = sys1.simulate(each_sim, withdraw_traj=False, verbosity=False)
            cur_state2 = sys2.simulate(each_sim, withdraw_traj=False, verbosity=False)

            if(MC_exchange(sys1, sys2)):
                sys1._currentPosition = cur_state2.position
                sys2._currentPosition = cur_state1.position
            else:
                pass

        s_val_posDict.update({sys2.potential.s[0] : sys2.trajectory})
        s_val_posDict.update({sys1.potential.s[0] : sys1.trajectory})

        end_sim = datetime.datetime.now()
        duration_sim = end_sim-start_sim

        #plotting:
        print("ploting")
        start_plot = datetime.datetime.now()
        fig = exPlot.plot_2D_2State_EDS_potential_sDependency(s_val_posDict, eds_pot=edsPot1, 
                                                              plot_trajs=True, space_range=simulation_space,
                                                              out_path=replica_out+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT_withTraj.png")
        end_plot = datetime.datetime.now()
        duration_plot = end_plot-start_plot

        for s in s_val_posDict:
            write_out_etraj(traj=s_val_posDict[s], out_path=replica_out+"/replica_traj_s"+str(s)+".dat", 
                            V1=sys1.potential.V_is[0], V2=sys1.potential.V_is[1])
        #del sys1, sys2
        break


    end_t = datetime.datetime.now()
    duration = end_t-start_t
    print("Done - duration: ", duration)
    print("Done - simulation duration: ", duration_sim)
    print("Done - plotting duration: ", duration_plot)

print(root_tmp_dir)

C:\Users\benja\AppData\Local\Temp/edsSim/re_eds_simulation



ploting

Done - duration:  8:11:10.469756
Done - simulation duration:  8:10:51.937266
Done - plotting duration:  0:00:13.964369



ploting

Done - duration:  6:06:58.190658
Done - simulation duration:  6:06:05.320788
Done - plotting duration:  0:00:43.371315



ploting

Done - duration:  16:02:45.866263
Done - simulation duration:  16:02:21.288339
Done - plotting duration:  0:00:19.516533


KeyboardInterrupt: 

In [ ]:
print(root_tmp_dir)


In [ ]:
sys1

In [ ]:
sys1.trajectory.head()